# Projet OT5: Parallélisation de micro-aevol

## Adrien Jaillet - Novembre 2022

### Présentation des fichiers

Le dossier est composé de plusieurs sous-dossier. Chaque sous-dossier correspon à une nouvelle version de micro-aevol. La version v0 correspond à la version initial. 

J'ai choisi d'axer ma parallélisation sur le GPU, car c'est ce qui m'interessait le plus.

### Configuration

Afin que les programmes puissent compiler sur votre machine, vous devez modifier le fichier .env à la racine du projet et mettre le chemin absolu de votre projet dans la variable "MICRO_AEVOL_PROJECT_DIRECTORY").

Vous devez ensuite executer la commande suivante:

In [4]:
!. .env

In [6]:
import os
import shutil
import subprocess
import sys

NUM_STEPS = 1000


def prRed(skk): print("\033[91m {}\033[00m" .format(skk))

def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))


def main():
    NUM_STEPS = 1000

    cwd = os.getcwd()
    
    print(cwd)
    if os.path.exists("experiment_cpu_v0"):
        shutil.rmtree("experiment_cpu_v0")

    if os.path.exists("experiment_to_challenge"):
        shutil.rmtree("experiment_to_challenge")

 

main()

/home/ajaillet/.local/share/Trash/files/result_comparison/experiment_cpu_v0
